In [39]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

In [40]:
data = pd.read_csv(r"C:\Users\soumya kemidi\OneDrive\Desktop\fake_news.csv")
data.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [41]:
data.shape

(20800, 5)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [43]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [47]:
data = data.drop(['id'], axis=1)

KeyError: "['id'] not found in axis"

In [48]:
data = data.fillna('')

In [49]:
data['content'] = data['author']+' '+ data['title']+' '+data['text']

In [50]:
data = data.drop(['title','author', 'text'], axis=1)

In [51]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [52]:
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [53]:
data['content'] = data['content'].str.replace('[^\w\s]','')

C:\Users\soumya kemidi\AppData\Local\Temp\ipykernel_28492\3643324700.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('[^\w\s]','')


In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\soumya
[nltk_data]     kemidi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [56]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

0    darrell lucus house dem aide didnt even see co...
1    daniel j flynn flynn hillary clinton big woman...
2    consortiumnewscom truth might get fired truth ...
3    jessica purkiss 15 civilian killed single u ai...
4    howard portnoy iranian woman jailed fictional ...
Name: content, dtype: object

In [57]:
X = data[['content']]
y = data['label']

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=45, stratify=y)

In [60]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfidf = tfidf_vect.transform(X_train['content'])
xtest_tfidf = tfidf_vect.transform(X_test['content'])

In [63]:
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn import metrics

pclf = PassiveAggressiveClassifier()

pclf.fit(xtrain_tfidf, y_train)

predictions = pclf.predict(xtest_tfidf)

print(metrics.classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.96      0.96      0.96      3116
           1       0.96      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [64]:
print(metrics.confusion_matrix(y_test,predictions))

[[2982  134]
 [ 122 3002]]


In [65]:
from sklearn.neural_network import MLPClassifier

mlpclf = MLPClassifier(hidden_layer_sizes=(256,64,16),

                       activation = 'relu', 

                       solver = 'adam')

mlpclf.fit(xtrain_tfidf, y_train)

predictions = mlpclf.predict(xtest_tfidf)

print(metrics.classification_report(y_test, predictions))






              precision    recall  f1-score   support

           0       0.96      0.95      0.96      3116
           1       0.95      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [66]:
print(metrics.confusion_matrix(y_test,predictions))






[[2968  148]
 [ 122 3002]]


In [67]:
import pickle

# Save trained model to file

pickle.dump(mlpclf, open("fakenews1.pkl", "wb"))

In [68]:
def fake_news_det(news):

    input_data = [news]

    vectorized_input_data = tfidf_vect.transform(input_data)

    prediction = pclf.predict(vectorized_input_data)

    print(prediction)






In [69]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')






[1]


In [70]:
fake_news_det(""" President Barack Obama has been campaigning hard for the 

woman who is supposedly going to extend his legacy four more years. 

The only problem with stumping for Hillary Clinton, however, is she is not 

exactly a candidate easy to get too enthused about.  """)






[1]


In [71]:
from flask import Flask, render_template, request

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import PassiveAggressiveClassifier

import pickle

import pandas as pd

from sklearn.model_selection import train_test_split

In [72]:
app = Flask(_name_)

tfvect = TfidfVectorizer(stop_words='english', max_df=0.7)

loaded_model = pickle.load(open('fakenews1.pkl', 'rb'))

dataframe = pd.read_csv('fake_news.csv')

x = dataframe['text']

y = dataframe['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

NameError: name '_name_' is not defined

In [ ]:
def fake_news_det(news):

    tfid_x_train = tfvect.fit_transform(x_train)

    tfid_x_test = tfvect.transform(x_test)

    input_data = [news]

    vectorized_input_data = tfvect.transform(input_data)

    prediction = loaded_model.predict(vectorized_input_data)

    return prediction

In [ ]:
@app.route('/')

def home():

    return render_template('index.html')

 

@app.route('/predict', methods=['POST'])

def predict():

    if request.method == 'POST':

        message = request.form['message']

        pred = fake_news_det(message)

        print(pred)

        return render_template('index.html', prediction=pred)

    else:

        return render_template('index.html', prediction="Ops, Something went wrong")

 

if _name_ == '_main_':

    app.run(debug=True)

In [75]:
# Ensure Flask is imported in this cell
from flask import Flask, render_template, request

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import PassiveAggressiveClassifier

import pickle

import pandas as pd

from sklearn.model_selection import train_test_split

app = Flask(__name__) # Changed _name_ to __name__

tfvect = TfidfVectorizer(stop_words='english', max_df=0.7)

loaded_model = pickle.load(open('fakenews1.pkl', 'rb'))

dataframe = pd.read_csv('fake_news.csv')

x = dataframe['text']

y = dataframe['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [76]:
def fake_news_det(news):

    tfid_x_train = tfvect.fit_transform(x_train)

    tfid_x_test = tfvect.transform(x_test)

    input_data = [news]

    vectorized_input_data = tfvect.transform(input_data)

    prediction = loaded_model.predict(vectorized_input_data)

    return prediction